# Step 1: Load Data

In [0]:
import pandas as pd
import numpy as np
import keras
from sklearn.model_selection import train_test_split

dataset_path = 'data/fer2013.csv'
num_classes = 7 #angry, disgust, fear, happy, sad, surprise, neutral
width, height = 48, 48

def load_data():
    data = pd.read_csv(dataset_path)
    pixels = data['pixels'].tolist()
    faces = []
    for pixel_sequence in pixels:
        pixels_1D = pixel_sequence.split(' ')
        pixels_2D = np.asarray(pixels_1D).reshape(width, height)
        face = np.array(pixels_2D, 'float32')  #shape = (48,48)
        faces.append(face)                     #shape = (35887, 48, 48)
    faces = np.expand_dims(faces, -1)          #shape = (35887, 48, 48, 1)  grayscale is 1 only; while RGB is 3
    faces = faces / 255                        # Normalized
    emotions = keras.utils.to_categorical(data['emotion'], num_classes) #one hot encode 7 categories
    return faces, emotions


faces, emotions = load_data()

### Remarks

pixels_1D
[70.,80.,82.,72.,58.,...]

pixels_2D
[array([[ 70.,  80.,  82., ...,  52.,  43.,  41.],
        [ 65.,  61.,  58., ...,  56.,  52.,  44.],
        [ 50.,  43.,  54., ...,  49.,  56.,  47.],
        ...,
        [ 91.,  65.,  42., ...,  72.,  56.,  43.],
        [ 77.,  82.,  79., ..., 105.,  70.,  46.],
        [ 77.,  72.,  84., ..., 106., 109.,  82.]], dtype=float32)]
        
Normalized  
array([[0.27450982, 0.3137255 , 0.32156864, ..., 0.20392157, 0.16862746,
        0.16078432],
       [0.25490198, 0.23921569, 0.22745098, ..., 0.21960784, 0.20392157,
        0.17254902],
       [0.19607843, 0.16862746, 0.21176471, ..., 0.19215687, 0.21960784,
        0.18431373],
       ...,
       [0.35686275, 0.25490198, 0.16470589, ..., 0.28235295, 0.21960784,
        0.16862746],
       [0.3019608 , 0.32156864, 0.30980393, ..., 0.4117647 , 0.27450982,
        0.18039216],
       [0.3019608 , 0.28235295, 0.32941177, ..., 0.41568628, 0.42745098,
        0.32156864]], dtype=float32)
        
Emotions
array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

# Step 2: Define Model

In [0]:
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.layers import Activation, Convolution2D, Dropout, Conv2D
from keras.layers import AveragePooling2D, BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras.models import Sequential
from keras.layers import Flatten
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import MaxPooling2D
from keras.layers import SeparableConv2D
from keras import layers
from keras.regularizers import l2
import pandas as pd
import cv2
import numpy as np
 
# parameters
patience = 50
input_shape = (48, 48, 1)
batch_size = 32
num_classes = 7
num_epochs = 8
data_augmentation = True
base_path = 'models/'

# model parameters
#regularization = l2(l2_regularization)
 
model = Sequential()

model.add(Conv2D(64, (5, 5), padding='same',
                 input_shape=faces.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(64, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

#model = Model(img_input, output)
#model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()
 
# callbacks
log_file_path = base_path + '_emotion_training.log'
csv_logger = CSVLogger(log_file_path, append=False)
early_stop = EarlyStopping('val_loss', patience=patience)
reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1, patience=int(patience/4), verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 64)        1664      
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 44, 44, 64)        102464    
_________________________________________________________________
activation_2 (Activation)    (None, 44, 44, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        36928     
_________________________________________________________________
activation_3 (Activation)    (None, 14, 14, 64)        0         
__________

# Step 3: Generator and Train/Test Splitting

In [0]:
data_generator = ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)

x_train, x_test,y_train,y_test = train_test_split(faces, emotions,test_size=0.2,shuffle=True)
train_generator = data_generator.flow(x_train, y_train, batch_size=batch_size)

# Step 4: Cross Validation Model Training

In [0]:
from sklearn.model_selection import KFold
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

kf = KFold(n_splits=5)
kf.get_n_splits(faces)

index = 1;
patience = 50
input_shape = (48, 48, 1)
batch_size = 32
num_classes = 7
num_epochs = 8
evaluation_result_accuracy = []
evaluation_result_precision = []
evaluation_result_recall = []
evaluation_result_fscore = []
evaluation_result_support = []


for train_index, test_index in kf.split(faces):
    print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = faces[train_index], faces[test_index]
    y_train, y_test = emotions[train_index], emotions[test_index]
    # callbacks
    log_file_path = base_path + '_emotion_training.log'
    csv_logger = CSVLogger(log_file_path, append=False)
    early_stop = EarlyStopping('val_loss', patience=patience)
    reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1, patience=int(patience/4), verbose=1)
    trained_models_path = base_path + 'model_' + str(index)
    model_names = trained_models_path + '.hdf5'
    model_checkpoint = ModelCheckpoint(model_names, 'val_loss', verbose=1,save_best_only=True)
    callbacks = [model_checkpoint, csv_logger, early_stop, reduce_lr]
    train_generator = data_generator.flow(x_train, y_train, batch_size=batch_size)
    
    model.summary()
    model.fit_generator(
                train_generator, 
                steps_per_epoch=len(x_train) / batch_size, 
                epochs=num_epochs, verbose=1, callbacks=callbacks,
                validation_data=(x_test,y_test)) #train for randomly selected one
    index += 1;
    y_pred_prob=model.predict(x_test)
    y_pred = []
    for i in range(y_pred_prob.shape[0]):
        y_result = [0] * 7
        y_result[np.argmax(y_pred_prob[i])] = 1
        y_pred.append(y_result)
    y_pred = np.array(y_pred)
    evaluation_result = precision_recall_fscore_support(y_test, y_pred)
    evaluation_result_accuracy.append(accuracy_score(y_test, y_pred))
    evaluation_result_precision.append(evaluation_result[0])
    evaluation_result_recall.append(evaluation_result[1])
    evaluation_result_fscore.append(evaluation_result[2])
    evaluation_result_support.append(evaluation_result[3])

TRAIN: [ 7178  7179  7180 ... 35884 35885 35886] TEST: [   0    1    2 ... 7175 7176 7177]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 64)        1664      
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 44, 44, 64)        102464    
_________________________________________________________________
activation_2 (Activation)    (None, 44, 44, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        36928     
___________________________________________________

TRAIN: [    0     1     2 ... 35884 35885 35886] TEST: [14356 14357 14358 ... 21530 21531 21532]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 64)        1664      
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 44, 44, 64)        102464    
_________________________________________________________________
activation_2 (Activation)    (None, 44, 44, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        36928     
_____________________________________________

898/897 [==============================] - 538s 599ms/step - loss: 0.8945 - acc: 0.6646 - val_loss: 0.7256 - val_acc: 0.7433

Epoch 00001: val_loss improved from inf to 0.72557, saving model to models/model_5.hdf5
Epoch 2/8
898/897 [==============================] - 537s 598ms/step - loss: 0.8854 - acc: 0.6654 - val_loss: 0.7140 - val_acc: 0.7413

Epoch 00002: val_loss improved from 0.72557 to 0.71401, saving model to models/model_5.hdf5
Epoch 3/8
898/897 [==============================] - 537s 598ms/step - loss: 0.8843 - acc: 0.6708 - val_loss: 0.7094 - val_acc: 0.7399

Epoch 00003: val_loss improved from 0.71401 to 0.70938, saving model to models/model_5.hdf5
Epoch 4/8
898/897 [==============================] - 537s 598ms/step - loss: 0.8801 - acc: 0.6718 - val_loss: 0.7040 - val_acc: 0.7401

Epoch 00004: val_loss improved from 0.70938 to 0.70397, saving model to models/model_5.hdf5
Epoch 5/8
898/897 [==============================] - 538s 599ms/step - loss: 0.8811 - acc: 0.6689 - va

In [0]:
for i in range(len(evaluation_result_accuracy)):
    print('Accuracy of ', i, ' is: ', evaluation_result_accuracy[i], '\n',
          'precision, recall, F-score and support for model  of ', i, ' are', '\n',
          evaluation_result_precision[i], '\n',
          evaluation_result_recall[i], '\n',
          evaluation_result_fscore[i], '\n',
          evaluation_result_support[i], '\n'
         )
print('The average accuracy for models is: ', 
          np.mean(evaluation_result_accuracy), '\n',
          'precision, recall, F-score and support', '\n',
          np.mean(evaluation_result_precision, axis=0), '\n',
          np.mean(evaluation_result_recall, axis=0), '\n',
          np.mean(evaluation_result_fscore, axis=0), '\n',
          np.mean(evaluation_result_support, axis=0)
         )

# Step 5: Model Training

In [0]:
from sklearn.model_selection import KFold
from sklearn.metrics import precision_recall_fscore_support

num_epochs = 100
trained_models_path = base_path + 'model_AlexNet_Full_Train'
model_names = trained_models_path + '.hdf5'
model_checkpoint = ModelCheckpoint(model_names, 'val_loss', verbose=1,save_best_only=True)
callbacks = [model_checkpoint, csv_logger, early_stop, reduce_lr]
train_generator = data_generator.flow(faces, emotions, batch_size=batch_size)
model.fit_generator(
                train_generator, 
                steps_per_epoch=len(x_train) / batch_size, 
                epochs=num_epochs, verbose=1, callbacks=callbacks,
                validation_data=(x_test,y_test)) #train for randomly selected one

Epoch 1/100
898/897 [==============================] - 624s 695ms/step - loss: 1.8235 - acc: 0.2483 - val_loss: 1.8054 - val_acc: 0.2484

Epoch 00001: val_loss improved from inf to 1.80539, saving model to models/model_AlexNet_Full_Train.hdf5
Epoch 2/100
898/897 [==============================] - 605s 673ms/step - loss: 1.7952 - acc: 0.2541 - val_loss: 1.7680 - val_acc: 0.2581

Epoch 00002: val_loss improved from 1.80539 to 1.76804, saving model to models/model_AlexNet_Full_Train.hdf5
Epoch 3/100
898/897 [==============================] - 579s 645ms/step - loss: 1.7684 - acc: 0.2704 - val_loss: 1.7352 - val_acc: 0.2917

Epoch 00003: val_loss improved from 1.76804 to 1.73519, saving model to models/model_AlexNet_Full_Train.hdf5
Epoch 4/100
898/897 [==============================] - 562s 626ms/step - loss: 1.7490 - acc: 0.2832 - val_loss: 1.6962 - val_acc: 0.3235

Epoch 00004: val_loss improved from 1.73519 to 1.69622, saving model to models/model_AlexNet_Full_Train.hdf5
Epoch 5/100
898/

898/897 [==============================] - 520s 579ms/step - loss: 1.2057 - acc: 0.5445 - val_loss: 1.0842 - val_acc: 0.5893

Epoch 00036: val_loss improved from 1.10422 to 1.08417, saving model to models/model_AlexNet_Full_Train.hdf5
Epoch 37/100
898/897 [==============================] - 520s 580ms/step - loss: 1.2039 - acc: 0.5428 - val_loss: 1.0825 - val_acc: 0.5938

Epoch 00037: val_loss improved from 1.08417 to 1.08251, saving model to models/model_AlexNet_Full_Train.hdf5
Epoch 38/100
898/897 [==============================] - 520s 579ms/step - loss: 1.1953 - acc: 0.5449 - val_loss: 1.0794 - val_acc: 0.5963

Epoch 00038: val_loss improved from 1.08251 to 1.07942, saving model to models/model_AlexNet_Full_Train.hdf5
Epoch 39/100
898/897 [==============================] - 520s 579ms/step - loss: 1.1831 - acc: 0.5540 - val_loss: 1.0617 - val_acc: 0.6014

Epoch 00039: val_loss improved from 1.07942 to 1.06169, saving model to models/model_AlexNet_Full_Train.hdf5
Epoch 40/100
898/897 

898/897 [==============================] - 521s 581ms/step - loss: 1.0351 - acc: 0.6105 - val_loss: 0.9027 - val_acc: 0.6620

Epoch 00072: val_loss improved from 0.90385 to 0.90274, saving model to models/model_AlexNet_Full_Train.hdf5
Epoch 73/100
898/897 [==============================] - 522s 582ms/step - loss: 1.0252 - acc: 0.6135 - val_loss: 0.9114 - val_acc: 0.6570

Epoch 00073: val_loss did not improve from 0.90274
Epoch 74/100
898/897 [==============================] - 522s 581ms/step - loss: 1.0264 - acc: 0.6140 - val_loss: 0.9207 - val_acc: 0.6453

Epoch 00074: val_loss did not improve from 0.90274
Epoch 75/100
898/897 [==============================] - 522s 582ms/step - loss: 1.0379 - acc: 0.6113 - val_loss: 0.8902 - val_acc: 0.6654

Epoch 00075: val_loss improved from 0.90274 to 0.89024, saving model to models/model_AlexNet_Full_Train.hdf5
Epoch 76/100
898/897 [==============================] - 562s 626ms/step - loss: 1.0271 - acc: 0.6127 - val_loss: 0.8890 - val_acc: 0.6647